In [2]:
import time
import random
import selenium

from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains






In [3]:
username = "K.Dixon1@lse.ac.uk"
password = ""

In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import json

def scrape_exam_papers(driver, start_box):
    list = []

    def get_elements(css_selector, retries=3):
        for attempt in range(retries):
            try:
                return WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, css_selector)))
            except StaleElementReferenceException:
                if attempt < retries - 1:
                    continue
                else:
                    raise

    def get_element(css_selector, retries=3):
        for attempt in range(retries):
            try:
                return WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector)))
            except StaleElementReferenceException:
                if attempt < retries - 1:
                    continue
                else:
                    raise

    def click_load_more():
        try:
            load_more_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/primo-explore/div/prm-explore-main/ui-view/prm-collection-discovery/prm-collection-gallery/md-content/div[2]/prm-gallery-items-list/div/div[3]/button")))
            load_more_button.click()
            time.sleep(1)
            print("Clicked 'Load More' button.")
        except NoSuchElementException:
            print("'Load More' button not found. Continuing...")

    try:
        driver.execute_script("document.body.style.zoom='30%'")
        boxes = get_elements(".margin-bottom-small > prm-gallery-collection")

        for m in range(start_box, len(boxes)):
            try:
                # Re-find the boxes element to avoid stale reference
                driver.execute_script("document.body.style.zoom='30%'")
                boxes = get_elements(".margin-bottom-small > prm-gallery-collection")
                box = boxes[m]
                details = {}

                # Extract title and URL
                title = get_element(f".margin-bottom-small > prm-gallery-collection:nth-child({m+1}) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(2) > span:nth-child(1)").text
                subject_page_url = box.find_element(By.TAG_NAME, "a").get_attribute("href")
                box.click()

                #if m == 0,1 2 4, or 23 then click load more enough times unitl the button no longer exissts, else continue to scrape
                if m in [1,3,4,5,6,7,8,10,11,12,13,14,15,16,19,23,24]:
                    while True:
                        click_load_more()
                        try: #if there is only one load more, then break the loop
                            load_more_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/primo-explore/div/prm-explore-main/ui-view/prm-collection-discovery/prm-collection-gallery/md-content/div[2]/prm-gallery-items-list/div/div[3]/button")))
                        except TimeoutException:
                            break




                containers = get_elements(f".is-grid-view > prm-gallery-item")

                for i in range(len(containers)):
                    # Re-find the containers element to avoid stale reference
                    containers = get_elements(f".is-grid-view > prm-gallery-item")
                    driver.execute_script("arguments[0].scrollIntoView();", containers[i])  # Scroll to the button
                    print(len(containers))
                    container = containers[i]

                    course_name = container.find_element(By.CLASS_NAME, "item-title").text
                    type = container.find_element(By.CSS_SELECTOR, "div:nth-child(2) > div:nth-child(2) > span:nth-child(1) > span:nth-child(1)").text
                    container.click()

                    exams = get_elements("md-list-item.md-3-line")
                    
                    for n in range(1, len(exams)+1):
                        print(len(exams))
                        exam = get_element(f"md-list-item.md-3-line:nth-child({n}) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)")
                        driver.execute_script("arguments[0].scrollIntoView();", exam)  # Scroll to the button
                        exam_year = exam.text
                        pdf_link = exam.get_attribute("href")

                        details = {
                            "name": title,
                            "subject_page_url": subject_page_url,
                            "course_name": course_name,
                            "type": type,
                            "exam_year": exam_year,
                            "pdf_link": pdf_link
                        }
                        list.append(details)
                        print(details)
                        

                    exit_button = get_element("button.md-icon-button:nth-child(4)")
                    exit_button.click()

                    
            except (StaleElementReferenceException, TimeoutException) as e:
                print(f"Container processing error: {e}")
                continue
                            

            #save current list to json file
            with open(f"exam_papers_{m}.json", "w") as f:
                json.dump(list, f, indent=4)
                print(f"Saved to exam_papers_{m}.json")
            back_button = get_element("prm-collection-navigation-breadcrumbs-item.layout-align-start-center:nth-child(2) > a:nth-child(1)")
            driver.execute_script("arguments[0].scrollIntoView();", back_button) 
            time.sleep(5) # Scroll to the button
            back_button.click()

            time.sleep(15)
            WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".margin-bottom-small > prm-gallery-collection")))


            
    except (StaleElementReferenceException, TimeoutException) as e:
        print(f"Container processing error: {e}")

    return list

In [68]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ...

driver = webdriver.Firefox()
driver.get('https://www.lse.ac.uk/library')

# enter full screen
driver.fullscreen_window()

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'Loginto'))).click()

# Let the user actually see something!
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[3]/div/div/h3[1]/a'))).click()

# Let the user actually see something!
WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.NAME, 'loginfmt'))).send_keys(username)

# Let the user actually see something!
next_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'idSIButton9')))
if next_button:
    next_button.click()
else:
    print("No next button found")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.NAME, 'passwd'))).send_keys(password)

# Let the user actually see something!
next_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'idSIButton9')))
next_button.click()

# Wait until the screen changes, this can be a variable length of time though
try:
    # Wait until the element is visible
    element = WebDriverWait(driver, 300).until(EC.visibility_of_element_located((By.ID, 'idBtn_Back')))
    # Act on the element as soon as it becomes visible
    element.click()
except TimeoutException:
    print("The element did not appear within the time limit")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.grid-item:nth-child(1) > button:nth-child(1)'))).click()

new_url = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '#mainMenu > div:nth-child(4) > a:nth-child(1)')))[0].get_attribute("href")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mainMenu > div:nth-child(4) > a:nth-child(1)'))).click()

#the driver must now be on the new page

WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
new_window = driver.window_handles[-1]
driver.switch_to.window(new_window)






In [73]:
scrape_exam_papers(driver, 0)

KeyboardInterrupt: 

In [63]:
scrape_exam_papers(driver, 1)


Clicked 'Load More' button.
45
2
{'name': 'Anthropology', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317130002021&lang=en', 'course_name': 'AN100 - Introduction to Social Anthropology', 'type': 'EXAM PAPER', 'exam_year': '2017', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235314550002021'}
2
{'name': 'Anthropology', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317130002021&lang=en', 'course_name': 'AN100 - Introduction to Social Anthropology', 'type': 'EXAM PAPER', 'exam_year': '2018', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235314560002021'}
45
5
{'name': 'Anthropology', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317130002021&lang=en', 'course_name

[{'name': 'Anthropology',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317130002021&lang=en',
  'course_name': 'AN100 - Introduction to Social Anthropology',
  'type': 'EXAM PAPER',
  'exam_year': '2017',
  'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235314550002021'},
 {'name': 'Anthropology',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317130002021&lang=en',
  'course_name': 'AN100 - Introduction to Social Anthropology',
  'type': 'EXAM PAPER',
  'exam_year': '2018',
  'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235314560002021'},
 {'name': 'Anthropology',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317130002021&lang=en',
  'course_name': 'AN101 -

In [77]:
scrape_exam_papers(driver, 4)

Clicked 'Load More' button.
Clicked 'Load More' button.
72
5
{'name': 'Economics', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317100002021&lang=en', 'course_name': 'EC100 - Economics A', 'type': 'EXAM PAPER', 'exam_year': '2017', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235367090002021'}
5
{'name': 'Economics', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317100002021&lang=en', 'course_name': 'EC100 - Economics A', 'type': 'EXAM PAPER', 'exam_year': '2017 Lent Term', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235367040002021'}
5
{'name': 'Economics', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317100002021&lang=en', 'course_name': 'EC100 - Economics 

ElementClickInterceptedException: Message: Element <prm-gallery-item> is not clickable at point (912,402) because another element <span> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:337:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:177:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:136:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:205:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:85:31


In [78]:
scrape_exam_papers(driver, 5)

Clicked 'Load More' button.
33
2
{'name': 'European Institute', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317080002021&lang=en', 'course_name': 'EU420 - European Law and Government', 'type': 'EXAM PAPER', 'exam_year': '2017', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235413040002021'}
2
{'name': 'European Institute', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317080002021&lang=en', 'course_name': 'EU420 - European Law and Government', 'type': 'EXAM PAPER', 'exam_year': '2018', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235413050002021'}
33
5
{'name': 'European Institute', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317080002021&lang=en', 'course_na

KeyboardInterrupt: 